## This ipynb show how to transfer the CONLL NER dataset to the backend's DB.

In [2]:
import pandas as pd

In [55]:
df = pd.read_csv("./NER_multilabel_data_v2.csv")

In [118]:
df[90:100]

,Unnamed: 0,Sentence #,Word,POS,Tag,newTag
90,90,Sentence: 5,the,DT,O,O
91,91,Sentence: 5,annual,JJ,O,O
92,92,Sentence: 5,conference,NN,O,O
93,93,Sentence: 5,of,IN,O,O
94,94,Sentence: 5,Britain,NNP,B-geo,Location|Party|B-geo
95,95,Sentence: 5,'s,POS,O,O
96,96,Sentence: 5,ruling,VBG,O,O
97,97,Sentence: 5,Labor,NNP,B-org,Party|B-org
98,98,Sentence: 5,Party,NNP,I-org,Party|I-org
99,99,Sentence: 5,in,IN,O,O


In [57]:
df.loc[:, "Sentence #"] = df["Sentence #"].fillna(method="ffill")

In [5]:
import numpy as np

In [10]:
sentences = list(set(df["Sentence #"]))

In [11]:
import requests

In [13]:
words = df[df["Sentence #"] == sentence]

In [14]:
def process_data(row):
    return {
        "text": row.Word,
        "labels": row.newTag.split("|")
    }

In [113]:
def post_data(sentence):
    data = {
        "user": "CONLL_0818@admin",
        "texts": list(df[df["Sentence #"] == sentence].apply(process_data, axis = 1))
    }
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json'
    }
    return requests.post("http://localhost:13537/data/label", data=json.dumps(data), headers = headers)

In [115]:
import json
from tqdm import tqdm
import threading
tasks = []
for sentence in tqdm(sentences):
    t = threading.Thread(target=post_data, args={sentence})
    t.start()
    tasks.append(t)
for task in tqdm(tasks):
    task.join()

100%|███████████████████████████| 47959/47959 [00:00<00:00, 529359.85it/s]
